In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from transformer_lens import utils

from sprint.loading import load_all

/opt/conda/envs/sprint/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model, data, sae = load_all()

Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([215402, 128]), dtype: torch.int64, device: cuda:0
Loading run1 from HuggingFace at 25
{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 2048,
 'dict_mult': 8,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'lr': 0.0001,
 'model_batch_size': 512,
 'num_tokens': 2000000000,
 'seed': 52,
 'seq_len': 128}
Encoder device: cuda:0


In [44]:
# prompt_clean = "I can't wait to see you again!"
# prompt_corrupted = "I really cant wait to see you again!"

prompt_clean = "They say that he doesn't"
prompt_corrupted = "Oh't say can you see"

# prompt_clean = "Full stop. It is in your best interest to comply with our request"
# prompt_corrupted = "I would really like to know how many tokens are in the other prompt"

# Check that the tokens line up
tokens_clean = model.to_tokens(prompt_clean).detach().cpu().numpy().flatten()
tokens_corrupted = model.to_tokens(prompt_corrupted).detach().cpu().numpy().flatten()

print("\t".join([model.to_string(t) for t in tokens_clean]), len(tokens_clean))
print("\t".join([model.to_string(t) for t in tokens_corrupted]), len(tokens_corrupted))

<|BOS|>	They	 say	 that	 he	 doesn	't 7
<|BOS|>	Oh	't	 say	 can	 you	 see 7


In [45]:
# Names
pattern = utils.get_act_name("attn_out", 0)
post = utils.get_act_name("post", 0)

# First, get the corrupt attention pattern
out_corrupt, cache_corrupt = model.run_with_cache(prompt_corrupted, names_filter=[pattern, post])
attn_corrupt = cache_corrupt[pattern]

# Then, get the clean outputs
out_clean, cache_clean = model.run_with_cache(prompt_clean, names_filter=[pattern, post])
attn_clean = cache_clean[pattern]


# Definte the ablation hook
def ablate_attn(activation, hook):
    print(hook.name)
    return attn_corrupt


# Now patch in the corrupt attention pattern and get outputs
with model.hooks(fwd_hooks=[(pattern, ablate_attn)]):
    out_resampled, cache_resampled = model.run_with_cache(prompt_clean, names_filter=[post])
# out_resampled = model.run_with_hooks(prompt_clean, fwd_hooks=[(pattern, ablate_attn), (post, cache_act)])

# Just for fun, the other way around:


def ablate_attn2(activation, hook):
    print(hook.name)
    return attn_clean


# out_resampled2 = model.run_with_hooks(prompt_corrupted, fwd_hooks=[(pattern, ablate_attn2), (post, cache_act2)])
# model.add_hook(pattern, ablate_attn2)
with model.hooks(fwd_hooks=[(pattern, ablate_attn2)]):
    out_resampled2, cache_resampled2 = model.run_with_cache(prompt_corrupted, names_filter=[post])
# model.remove_all_hook_fns()

blocks.0.hook_attn_out
blocks.0.hook_attn_out


In [46]:
FEAT = 10996
# TOK = 3
TOK = 6
# FEAT = 4542
# TOK = 5

def get_feature_mid_jacob(prompt, feature_token_idx, feature_post, use_ln=True, layer=0, mlp_out=True, model=None, encoder=None):
    with torch.no_grad():
        _, cache = model.run_with_cache(prompt, names_filter=[
          utils.get_act_name("resid_mid", layer)
        ])
    mid_acts = cache[utils.get_act_name("resid_mid", layer)]
    x_mid = mid_acts[0, feature_token_idx][None, None, :]

    my_fun = (ln2_mlp_until_post if not mlp_out else ln2_mlp_until_out)
    feature_mid = get_tangent_plane_at_point(x_mid,
        lambda x: my_fun(x, model.blocks[layer].ln2, model.blocks[layer].mlp, use_ln=use_ln),
        feature_post
    )[0,0]
    return feature_mid

for cache in [cache_corrupt, cache_clean, cache_resampled, cache_resampled2]:
    act = cache[post]
    feat = sae(act[0, TOK])[2].detach().cpu().numpy().flatten()
    print(feat[FEAT])

# Cosine similarities
caches = [cache_corrupt, cache_clean, cache_resampled, cache_resampled2]
print("\t".join(["cor", "clean", "clean->cor", "cor->clean"]))
for cache1 in [cache_corrupt, cache_clean, cache_resampled, cache_resampled2]:
    act1 = cache1[post]
    mid1 = get_feature_mid()
    for cache2 in [cache_corrupt, cache_clean, cache_resampled, cache_resampled2]:
        act2 = cache2[post]
        sim = torch.nn.functional.cosine_similarity(act1, act2, dim=-1)
        print(f"{sim[0, TOK].item():.3f}", end="\t")
    print()

0.0
1.859
1.292
0.0
cor	clean	clean->cor	cor->clean


NameError: name 'get_feature_mid' is not defined

In [ ]:
dla()